In [1]:
import cv2
import sys
import argparse
import numpy as np
import os
import multiprocessing
import tensorflow as tf
from tensorflow.contrib.layers import variance_scaling_initializer
from tensorpack import *
from tensorpack.utils.stats import RatioCounter
from tensorpack.tfutils.symbolic_functions import *
from tensorpack.tfutils.summary import *

In [ ]:
with tf.variable_scope('fc_regression') as scope:
                
                num_inputs = int(net.fc7_dropout.get_shape()[1])
                w, b = net.get_fc_weights(net_data=None,
                                          num_inputs=num_inputs,
                                          num_outputs=n_joints * 2,
                                          wights_std=0.01,
                                          bias_init_value=0.0)
                net.fc_regression = tf.nn.xw_plus_b(net.fc7_dropout, w, b, name=scope.name)

In [20]:
import pydot
print pydot.find_graphviz()
import graphviz

AttributeError: 'module' object has no attribute 'find_graphviz'

In [24]:
import resnet
from keras import backend as K
from keras.models import Model
model= resnet.ResnetBuilder.build_resnet_18((3, 227, 227), 3)
model = model.compile(loss='categorical_crossentropy', optimizer="adam")
outputs = [layer.output for layer in model.layers]                                            # input placeholder

AttributeError: 'tuple' object has no attribute 'compile'

In [12]:
DEPTH = 18

def shortcut(l, n_in, n_out, stride):
            if n_in != n_out:
                return Conv2D('convshortcut', l, n_out, 1, stride=stride)
            else:
                return l       
def basicblock(l, ch_out, stride, preact):
            ch_in = l.get_shape().as_list()[1]
            if preact == 'both_preact':
                l = BNReLU('preact', l)
                input = l
            elif preact != 'no_preact':
                input = l
                l = BNReLU('preact', l)
            else:
                input = l
            l = Conv2D('conv1', l, ch_out, 3, stride=stride, nl=BNReLU)
            l = Conv2D('conv2', l, ch_out, 3)
            return l + shortcut(input, ch_in, ch_out, stride)

        
def bottleneck(l, ch_out, stride, preact):
            ch_in = l.get_shape().as_list()[1]
            if preact == 'both_preact':
                l = BNReLU('preact', l)
                input = l
            elif preact != 'no_preact':
                input = l
                l = BNReLU('preact', l)
            else:
                input = l
            l = Conv2D('conv1', l, ch_out, 1, nl=BNReLU)
            l = Conv2D('conv2', l, ch_out, 3, stride=stride, nl=BNReLU)
            l = Conv2D('conv3', l, ch_out * 4, 1)
            return l + shortcut(input, ch_in, ch_out * 4, stride)
        
        

In [1]:
import tensorflow.contrib.slim as slim

In [13]:
def layer(l, layername, block_func, features, count, stride, first=False):
            with tf.variable_scope(layername):
                with tf.variable_scope('block0'):
                    l = block_func(l, features, stride,
                                   'no_preact' if first else 'both_preact')
                for i in range(1, count):
                    with tf.variable_scope('block{}'.format(i)):
                        l = block_func(l, features, 1, 'default')
                return l

In [18]:
cfg = {     18: ([2, 2, 2, 2], basicblock),
            34: ([3, 4, 6, 3], basicblock),
            50: ([3, 4, 6, 3], bottleneck),
            101: ([3, 4, 23, 3], bottleneck)
        }
defs, block_func = cfg[DEPTH]
with argscope(Conv2D, nl=tf.identity, use_bias=False,W_init=variance_scaling_initializer(mode='FAN_OUT')), \
                argscope([Conv2D, MaxPooling, GlobalAvgPooling, BatchNorm], data_format='NCHW'):
        logits = (LinearWrap(image)
                  .Conv2D('conv0', 64, 7, stride=2, nl=BNReLU)
                  .MaxPooling('pool0', shape=3, stride=2, padding='SAME')
                  .apply(layer, 'group0', block_func, 64, defs[0], 1, first=True)
                    .apply(layer, 'group1', block_func, 128, defs[1], 2)
                    .apply(layer, 'group2', block_func, 256, defs[2], 2)
                    .apply(layer, 'group3', block_func, 512, defs[3], 2)
                    .BNReLU('bnlast')
                    .GlobalAvgPooling('gap')
                    .FullyConnected('linear', 1000, nl=tf.identity)())   

NameError: name 'image' is not defined

In [21]:
 cfg = {
            18: ([2, 2, 2, 2], basicblock),
            34: ([3, 4, 6, 3], basicblock),
            50: ([3, 4, 6, 3], bottleneck),
            101: ([3, 4, 23, 3], bottleneck)
        }
print cfg[18]

([2, 2, 2, 2], <function basicblock at 0x7faf95f5eed8>)
